## LDPC + RL
This notebook will contain experiments conducted as part of the work done for ISIT 2024 submissions 

#### Experiment 1:
Comparing the Convergence of LLRs between Iterative decoder (flooding schedule) and Layered decoder.

In [1]:
%pylab inline 
import matlab
import numpy as np
import scipy as sp
import networkx as nx 
import matlab.engine
MATLAB = matlab.engine.start_matlab()
initial = 0

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


The parity check matrix used here is the 5G NR base matrix;

The `SubMatrix` is represented by:

In [2]:
if initial == 0:
    MATLAB.eval("addpath('./LDPC_M'); cd './LDPC_M/'; setup;",nargout=0)
    initial = 1

Z = int(MATLAB.workspace['z'])
h_submatrix = array(MATLAB.workspace['H'], dtype=int)
m,n = shape(h_submatrix)


option =

    'wlan'



In [3]:
def cyc_shift_iden(k,z):
    I = np.eye(z)
    I_1 = I[:, z-1].reshape((z,1))
    I_1 = np.append(I_1, I[:,0:z-1], axis=1)

    if k == -1:
        return np.zeros((z,z))
    elif k == 0:
        return I
    else: 
        return np.linalg.matrix_power(I_1,k)
    
    
def BitsinCheck(layer, z, row_num):
    bitnodes = []
    for SubMatrixCol, SubMatrixVal in enumerate(layer):
        if SubMatrixVal == -1:
            continue
        elif SubMatrixVal == 0:
            bitnodes.append(z*SubMatrixCol + ((row_num)%z))
        else:
            bitnodes.append(z*SubMatrixCol + ((row_num + SubMatrixVal-1)%z) + 1)
    
    return bitnodes


def incrementor(prev_bitnodes,z):
    next_bitnodes = []
    for b in prev_bitnodes:
        quotient, remainder = b//z, b%z
        next_pos = (remainder + 1)%z
        next_bitnodes.append(z*quotient + next_pos)
    
    return next_bitnodes


def min_algo(x):
    y = abs(x)
    min1 = np.min(y)
    min1_pos = np.argmin(y)

    y[min1_pos] = np.inf
    min2 = min(y)

    w = min1*np.ones(len(x),)
    w[min1_pos] = min2

    parity = np.prod(np.sign(x))
    w = w*parity*np.sign(x)

    return w         


def SubMatrix2PCM(SubMatrix, z):
    m,n = shape(SubMatrix)
    
    H = cyc_shift_iden(SubMatrix[0,0],z)
    for idx in range(1,n): 
        H = append(H, cyc_shift_iden(SubMatrix[0,idx],z),1)

    for i in range(1,m): 
        g = cyc_shift_iden(SubMatrix[i,0],z)
        for idx in range(1,n): 
            g = append(g, cyc_shift_iden(SubMatrix[i,idx],z),1)
        H = append(H,g,0)
    return H


def int_m(l):
    fin = 0
    for i in l:
        fin = 2*fin + i
    return fin

In [4]:
g = []
for val in h_submatrix[0,:].tolist():
    g += (cyc_shift_iden(val,81)[0,:]).tolist()
    
np.where(np.array(g)==1) , BitsinCheck(h_submatrix[0,:].tolist(), 81,0) 

((array([  57,  374,  497,  698,  889,  973, 1053]),),
 [57, 374, 497, 698, 889, 973, 1053])

In [5]:
H = SubMatrix2PCM(h_submatrix, Z)
M,N = shape(H)
h = sp.sparse.csr_array(H)
G = nx.bipartite.from_biadjacency_matrix(h)

In [6]:
K =  6
a = list(nx.simple_cycles(G,K))
a = [cyc for cyc in a if len(cyc) == K]
b = []
for cyc in a:
    if cyc[0] >= M:
        b.append(cyc[1:] + [cyc[0]])
    else:
        b.append(cyc)

In [7]:
len(b)
checjs = set()
for cyc in b:
    for node in cyc:
        checjs.add(node)

len(checjs)

2673

In [8]:
def Unionise(A):
    Union = set()
    for key,value_list in A.items():
        for value in value_list:
            Union.add(value)
    return Union 

def Histogramize(A,size):
    Hist = zeros((size,))
    Set_invert = {i:[] for i in range(size)}

    for key,value_list in A.items():
        for value in value_list:
            Set_invert[value].append(key)
            Hist[value] += 1
    
    return Hist, Set_invert

In [9]:
# Cycle maximised cluster formation
CN = set()
for i in range(M):
    CN.add(i)

def cluster_form(z, K_cycles, K,M):
    x = 1
    Clusters = {}
    e = 0
    C = {}

    for cyc in K_cycles:
        S_x = [cyc[i] for i in range(K) if not i%2]
        C.update({(x-1):S_x})
        x += 1

    Hist, Set_invert = Histogramize(C,M) 

    for cyc in K_cycles:
        # getting c_star such that:
            # c_star \in CNs \ all CNs already in clusters
            # c_star = argmax histogram(CNs \ all CNs already in clusters)

        Cluster_union = Unionise(Clusters)

        Remaining = CN - Cluster_union
        if len(Remaining) == 0:
            break        
        
        cn_list = [x for x in Remaining]
        c_star = cn_list[argmax(Hist[cn_list])]

        Union_SK = {}
        for k in Set_invert[c_star]:
            Union_SK.update({k:C[k]})

        _C_ = Unionise(Union_SK) - Cluster_union
        sorted_C_ = sorted(_C_)

        if len(_C_) >= z:
            new_cluster_list = [sorted_C_[x] for x in range(z)]
            Clusters.update({e:new_cluster_list})
            e += 1             
        else:
            _C_bar = sorted(CN - Cluster_union.union(_C_))
            _C_bar = set(_C_bar[:z-len(_C_)])
            new_cluster_list = sorted(_C_bar.union(_C_))
            Clusters.update({e:new_cluster_list})
            e += 1

    return Clusters

In [10]:
# Clustering check 

clusters = cluster_form(81, b, 6, M)

In [11]:
clust_list = [clusters[i] for i in range(len(clusters))]

MATLAB.workspace['cluster_alloc'] = matlab.int64(clust_list)
MATLAB.eval("cluster_alloc = cluster_alloc + 1;", nargout=0)

# running the script for cluster allocated layers
# MATLAB.eval("cd LDPC_dir",nargout=0)

MATLAB.eval("save('Imp.mat')",nargout=0)

# MATLAB.ldpc_cluster_layered(nargout=0)
# # MATLAB.eval("size(cluster_alloc)")

# MATLAB.eval("P_ecw")
# MATLAB.eval("new_indices =[]; for layer = 1:numSubMatrixRows new_indices = [new_indices, cluster_alloc(layer,:)]; end ", nargout=0)
# MATLAB.eval("H(new_indices,:)")

In [ ]:
def NeighborVN(Cluster):
    temp = set()
    for cn in Cluster:
        cns_vns = array(G[cn]) - M
        for vn in cns_vns:
            temp.add(vn)
    
    return sorted(temp)

#### RELDEC Algorithm 

prerequisites:
-   Cycle maximizing algorithm that allocates clusters of size `z` from `m` bitnodes ::tick!::
-   State space representation of CNs (and hence neighbouring VNs)

The plan is to keep the original indices intact so that the order of nodes through which states get updated can be kept constant. 

All the RL steps can be accomplished through python structures 


All the BP local flooding computations can be done on MATLAB scripts that support this notebook

In [ ]:
k_max = int(MATLAB.eval("max(sum(a~=-1, 2)')"))  # max number of varnodes per parity check eq  
z = 2
tau = int(ceil(m/z))
_l_max = 50
epsilon = 0.7

# clusters = cluster_form(81, b, 6, M)
clusters = cluster_form(z, b, 6, M)

A = [i for i in range(tau)] 

S = {}

Q = zeros((2**(z*k_max), tau))

# import L from matlab
MATLAB.eval("load('./LDPC_M/L_dataset.mat');", nargout =0)
L = array(matlab.double(MATLAB.workspace['L']))
C = array(matlab.double(MATLAB.workspace['C']))

ndata = shape(L)[0]

In [ ]:
# RELDEC 
for idx in range(ndata):
    l = L[idx,:]
    c = C[idx,:]

    _l = 0
    L_hat_l = l

    # initialize S
    
    for cluster_idx, cluster in clusters.items():
        S.update({cluster_idx: int_m(l[NeighborVN(cluster)] < 0)})

    while _l < _l_max:

        # happens with probability epsilon 
        if random.random() <= epsilon:
            a = np.random.randint(0,tau)
        else:
            a = argmax(Q[list(S.keys()), :], axis=0)
        
        cluster_a = clusters[a]
        vns_at_cluster = NeighborVN(cluster_a)
        l_a = len(vns_at_cluster)

        MATLAB.workspace['curr_clust'] = matlab.double(cluster_a)
        l_hat, CNarray, VNarray = MATLAB.local_flood(nargout=0)

        x_hat_a = (l_hat[vns_at_cluster] < 0)*1
        s_a = int_m(x_hat_a)
    
        Reward = (1/l_a)*sum(x_hat_a == c[vns_at_cluster])
        
        

        _l += 1